In [1]:
# imports
import huggingface as hf
import pandas as pd
import numpy as np
from datasets import load_dataset, concatenate_datasets, Dataset
import torch

In [62]:
# load hf dataset and convert to pandas dataframe
dataset = load_dataset("allenai/tulu-2.5-preference-data")
combined_dataset = concatenate_datasets([
    dataset["capybara"],
    dataset["ultrafeedback_evol_instruct"],
    dataset["ultrafeedback_truthful_qa"]
])
df = pd.DataFrame(combined_dataset)
# display first few rows
df.head()

chosen  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             [{'role': 'user', 'content': 'I'm debugging a memory leak in a node.js server application.

I have a couple heap dumps, explain how to debug this step by step.'}, {'role': 'assistant', 'content': 'Debugging a memory leak in a Node.js server application can be a challenging task. However, using heap dumps can significantly simplify the process. Here's a step-by-step guide on how to debug a memory leak using heap dumps:

1. Install necessary tools:

To analyze the heap dumps, you'll need the `heapdump` module to generate the dumps and `chrome-devtools-frontend` to visualize and inspect them.

Install the `heapdump` module:
```bash
npm install heapdump
```
1. Generate heap dumps:

Include the `heapdump` module in your server application and set up a way to trigger the heap dump, either on a certain condition or via an API endpoint. For example, you can add the following code to create an endpoint to generate a heap dump:
```javascript
const heapdump = require('heapdump');
const http = require('http');

http.createServer((req, res) => {
  if (req.url === '/heapdump') {
    const file = `/path/to/dumps/heapdump-${Date.now()}.heapsnapshot`;
    heapdump.writeSnapshot(file, (err, filename) => {
      if (err) {
        console.error(err);
        res.statusCode = 500;
        res.end('Error generating heap dump');
      } else {
        console.log(`Heap dump written to ${filename}`);
        res.statusCode = 200;
        res.end(`Heap dump written to ${filename}`);
      }
    });
  }
}).listen(3000);
```
Now, you can genera

In [63]:
# modify to have input, chosen and rejected as keys
# Function to extract input, chosen, rejected text
def extract_text(example):
    # Extract user content (input)
    chosen_user = next((m["content"] for m in example["chosen"] if m["role"] == "user"), "")
    rejected_user = next((m["content"] for m in example["rejected"] if m["role"] == "user"), "")

    # Usually user messages are identical in both — pick one
    user_input = chosen_user or rejected_user

    # Extract assistant content (responses)
    chosen_assistant = next((m["content"] for m in example["chosen"] if m["role"] == "assistant"), "")
    rejected_assistant = next((m["content"] for m in example["rejected"] if m["role"] == "assistant"), "")

    return {
        "input": user_input.strip(),
        "chosen": chosen_assistant.strip(),
        "rejected": rejected_assistant.strip(),
    }

# Apply to dataset
combined_dataset = combined_dataset.map(extract_text)

# Remove old columns if desired
combined_dataset = combined_dataset.remove_columns(["source"])

Map:   0%|          | 0/18374 [00:00<?, ? examples/s]

In [66]:
# convert to df
df = pd.DataFrame(combined_dataset)
# shuffle dataset
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
print(f"Total examples: {len(combined_dataset)}")

Total examples: 18374


In [67]:
# display first few rows
df.head()

,chosen,rejected,input
0,"To determine which aircraft is more cost-effective, we need to calculate the cost per passenger per mile for each aircraft.\n\nAircraft A:\nOperating cost per mile: $5\nPassengers: 150\n\nCost per passenger per mile = (Operating cost per mile) / Passengers\nCost per passenger per mile = $5 / 150 = $0.0333\n\nAircraft B:\nOperating cost per mile: $6\nPassengers: 200\n\nCost per passenger per mile = (Operating cost per mile) / Passengers\nCost per passenger per mile = $6 / 200 = $0.03\n\nSince Aircraft B has a lower cost per passenger per mile ($0.03) compared to Aircraft A ($0.0333), it would be more cost-effective for this route.","To determine which aircraft is more cost-effective, we need to calculate the cost per passenger per mile for each aircraft.\n\nAircraft A:\nOperating cost per mile: $5\nPassengers: 150\n\nCost per passenger per mile = (Operating cost per mile) / Passengers\nCost per passenger per mile = $5 / 150 = $0.0333\n\nAircraft B:\nOperating cost per mile: $6\nPassengers: 200\n\nCost per passenger per mile = (Operating cost per mile) / Passengers\nCost per passenger per mile = $6 / 200 = $0.03\n\nSince Aircraft B has a lower cost per passenger per mile ($0.03) compared to Aircraft A ($0.0333), it would be more cost-effective for this route.","An airline is considering adding a new route between two cities. The distance between the cities is 1,000 miles, and the airline has two types of aircraft available: Aircraft A has a range of 2,500 miles and can carry 150 passengers, while Aircraft B has a range of 1,800 miles and can carry 200 passengers. The operating cost per mile for Aircraft A is $5, and for Aircraft B it's $6. Determine which aircraft would be more cost-effective for this route, taking into consideration both passenger capacity and operating costs."
1,"A bus is typically heavier than a car. This is due to several factors, including size, weight of materials used, and seating capacity. On average, a car can weigh between 3,000 to 5,000 pounds (1,360 to 2,270 kilograms) depending on its make and model. On the other hand, a standard city bus can weigh between 25,000 to 40,000 pounds (11,340 to 18,144 kilograms). This difference in weight is primarily due to the larger size and increased passenger capacity of a bus, which requires stronger materials and additional support systems for structural integrity and safety.","A bus is generally heavier than a car. Cars come in various sizes and weights, but an average car weighs around 3,000 to 4,000 pounds (1,360 to 1,814 kg).",what is heavier a car or a bus?
2,"Predators:\n- Cheetah\n\nPrey:\n- Rabbit\n\nAlthough not a traditional predator, the elephant does not fit neatly into either category. Elephants primarily consume plants and are considered herbivores, but due to their massive size and strength, they have few natural predators, so they are neither typical predators nor prey themselves.","Cheetah - Predator\nElephant - Neither (Herbivore, not typically considered as prey)\nRabbit - Prey","Categorize these animals as predators/prey.\nCheetah, Elephant, Rabbit"
3,"1. Software Developer: Software developers use computers to design, develop, test, and maintain various types of software, including application software, systems software, and web applications. They rely on programming languages, debuggers, integrated development environments (IDEs), and other computer tools to create and optimize software for various platforms and users.\n\n2. Graphic Designer: Graphic designers use computers and specialized software to create visual concepts for different media channels. These professionals may work on projects such as designing logos, websites, advertisements, social media graphics, and more. They rely on programs like Adobe Photoshop, Illustrator, and InDesign to bring their artistic vision to life.\n\n3. Data Analyst: Data analysts use powerful computers and analytical software to collect, process, and interpret large data sets. T

# Prompt Classification

In [7]:
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



### classification using bart - not good

In [ ]:
from transformers import pipeline
from datasets import Dataset
from tqdm.auto import tqdm
import os

# Define labels and concise system prompt
labels = ["factual", "creative"]
system_prompt = """Classify the following query as either 'factual' or 'creative'. 'Factual' queries seek information about a particular fact. These include questions about events, definitions, explanations, factual queriesor data that can be verified or researched. 'Creative' queries request imaginative, creative content. These include prompts for storytelling, brainstorming, opinions, essay writing, etc.

Examples:
Query: What is the happiest country in the world?
Answer: factual

Query: Write a short story about a dragon and a knight.
Answer: creative

Query: """


# Initialize zero-shot classifier (use GPU if available)
classifier = pipeline(
    "zero-shot-classification",
    model="facebook/bart-large-mnli",
    device_map="auto",
    batch_size=32,          # tune based on GPU memory
    truncation=True
)

# # use llama-1b model for classification
# classifier = pipeline(
#     "text-classification",
#     model="meta-llama/Llama-3.2-1B",
#     device_map="auto",
#     batch_size=32,          # tune based on GPU memory
#     truncation=True, 
#     # token="hf_xxx"  # replace with your hf token if needed
# )

# Prepare batched function
def batch_classify(batch):
    inputs = [system_prompt + text + "\nAnswer: " for text in batch["input"]]
    # prompt a small LLM to classify
    results = classifier(inputs, candidate_labels=labels, multi_label=False)
    # store label, factual and creative scores
    batch["label"] = [res["labels"][0] for res in results]
    batch["factual_score"] = [
        res["scores"][res["labels"].index("factual")] for res in results
    ]
    batch["creative_score"] = [
        res["scores"][res["labels"].index("creative")] for res in results
    ]
    return batch

# convert pandas dataframe back to hf dataset
dataset = Dataset.from_pandas(df)
# # take first 2 samples
# dataset = dataset.select(range(60, 80))
# Parameters
save_dir = "../data/dpo_factual_creative_split_llm_version"
save_every = 640  # Save every 640 samples

# Create save directory if it doesn’t exist
os.makedirs(save_dir, exist_ok=True)

# Process dataset in chunks and save periodically
chunk_size = save_every
total = len(dataset)

for start in tqdm(range(0, total, chunk_size)):
    end = min(start + chunk_size, total)
    chunk = dataset.select(range(start, end))
    chunk = chunk.map(batch_classify, batched=True, batch_size=32)
    
    # Save chunk to disk
    chunk.save_to_disk(f"{save_dir}/chunk_{start}_{end}")
    print(f"✅ Saved chunk {start}–{end} to disk")

Some parameters are on the meta device because they were offloaded to the disk.
Device set to use mps


  0%|          | 0/27 [00:00<?, ?it/s]

Parameter 'function'=<function batch_classify at 0x5448d5d00> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only shown once. Subsequent hashing failures won't be shown.


Map:   0%|          | 0/640 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/640 [00:00<?, ? examples/s]

...

Saving the dataset (0/1 shards):   0%|          | 0/640 [00:00<?, ? examples/s]

✅ Saved chunk 16000–16640 to disk


Map:   0%|          | 0/607 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/607 [00:00<?, ? examples/s]

✅ Saved chunk 16640–17247 to disk


In [18]:
from datasets import concatenate_datasets
# merge all chunks back into a single dataset
all_chunks = []
for start in range(0, total, chunk_size):
    end = min(start + chunk_size, total)
    chunk = Dataset.load_from_disk(f"{save_dir}/chunk_{start}_{end}")
    all_chunks.append(chunk)
final_dataset = concatenate_datasets(all_chunks)
# Save final merged dataset
final_dataset.save_to_disk(f"{save_dir}/final_factual_creative_dataset")

print("🎉 All chunks processed and saved successfully!")

Saving the dataset (0/1 shards):   0%|          | 0/17247 [00:00<?, ? examples/s]

🎉 All chunks processed and saved successfully!


In [19]:
# load final merged dataset and display first few rows
final_dataset = Dataset.load_from_disk(f"{save_dir}/final_factual_creative_dataset")
final_df = final_dataset.to_pandas()
# display first 10 rows
# increase column display width
pd.set_option('display.max_colwidth', 6000)
final_df.head(10)

chosen  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    15.667   
1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             The dragon let out a deep, rumbling roar as it readied itself for what adventures awaited it.   
2                                                                      1. Create a customer loyalty program with rewards such as discounts and exclusive offers. \n2. Personalize customer service by learning customers’ names and offering assistance quickly and professionally. \n3. Provide outstanding customer service and quick personalized responses through various customer service channels.\n4. Develop an interactive blog or social media page to engage customers, answer questions and address customer feedback. \n5. Show appreciation and gratitude with surprise gifts and rewards for customers. \n6. Promote customer loyalty through referral programs and encourage customers to bring their friends and family. \n7. Surprise customers with personalized gifts on their birthday or anniversary of their first purchase. \n8. Offer same-day delivery or overnight shipping for expedited customer convenience.   
3                                                                                                                                                                                                                                                                                                                                                             Kaleem is an eight-year-old boy from India who has a very rare condition that makes his hands grow very big and heavy. His hands are so big that they now weigh over two stone and he has trouble doing simple tasks like tying his shoelaces or eating. His hands measure 13 inches from the base of his palm to the end of his middle finger. Kaleem's parents don't earn much money, so they are trying to raise funds to get the proper treatment for their son. Some doctors think he might have a condition called lymphangioma or hamartoma, which could be treatable.   
4  In 2020, the food industry faced numerous changes due to the ongoing global pandemic, including changes in consumer behavior, production cuts, and disruptions in supply chains. As a result, there was a lot of volatility within the industry as businesses struggled to adjust to the new reality. Demand for fresh and packaged goods declined, imp

In [ ]:
# get number of factual and creative samples
factual_count = sum(final_df['label'] == 'factual')
creative_count = sum(final_df['label'] == 'creative')
print(f"Factual samples: {factual_count}")
print(f"Creative samples: {creative_count}")

Factual samples: 14008
Creative samples: 3239


## retrying classification with google flan

In [82]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import Dataset
from tqdm.auto import tqdm
import os

# === Config ===
MODEL = "MBZUAI/LaMini-Flan-T5-248M"
labels = ["informational", "creative"]
system_prompt = """Classify the following user prompt as 'informational' or 'creative':"""

# === Load model & tokenizer ===
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL, device_map="auto", torch_dtype=torch.float16)

# === Classification function ===
def classify_with_probs(texts):
    results = []
    for text in texts:
        input_text = f"{system_prompt}{text}\nAnswer:"

        # Tokenize input once
        inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

        # Compute log-prob for each candidate label
        label_scores = {}
        with torch.no_grad():
            for label in labels:
                label_ids = tokenizer(label, return_tensors="pt").input_ids.to(model.device)
                outputs = model(**inputs, labels=label_ids)
                # Negative loss = log-probability
                logprob = -outputs.loss.item()
                label_scores[label] = logprob

        # Softmax for normalized probabilities
        probs = torch.softmax(torch.tensor(list(label_scores.values())), dim=0)
        factual_score, creative_score = probs.tolist()
        predicted_label = labels[int(torch.argmax(probs))]

        results.append({
            "label": "factual" if predicted_label == "informational" else "creative",
            "factual_score": factual_score,
            "creative_score": creative_score
        })
    return results


# === Batched map function ===
def batch_classify(batch):
    preds = classify_with_probs(batch["input"])
    batch["label"] = [p["label"] for p in preds]
    batch["factual_score"] = [p["factual_score"] for p in preds]
    batch["creative_score"] = [p["creative_score"] for p in preds]
    return batch

In [95]:
# === Dataset setup ===
# Assume df is your pandas dataframe with column "input"
dataset = Dataset.from_pandas(df)
# # take first 2 samples
# dataset = dataset.select(range(12, 76))

save_dir = "../data/dpo_informational_creative_split_lamini_version"
save_every = 320  # Save every 320 samples
os.makedirs(save_dir, exist_ok=True)
chunk_size = save_every
total = len(dataset)

for start in tqdm(range(0, total, chunk_size)):
    end = min(start + chunk_size, total)
    chunk = dataset.select(range(start, end))
    chunk = chunk.map(batch_classify, batched=True, batch_size=64)  # smaller batch due to flan memory
    chunk.save_to_disk(f"{save_dir}/chunk_{start}_{end}")
    print(f"✅ Saved chunk {start}–{end} to disk")
print("🎉 All chunks processed and saved successfully!")

  0%|          | 0/58 [00:00<?, ?it/s]

Map:   0%|          | 0/320 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (781 > 512). Running this sequence through the model will result in indexing errors


Saving the dataset (0/1 shards):   0%|          | 0/320 [00:00<?, ? examples/s]

✅ Saved chunk 0–320 to disk


...

Saving the dataset (0/1 shards):   0%|          | 0/320 [00:00<?, ? examples/s]

✅ Saved chunk 17920–18240 to disk


Map:   0%|          | 0/134 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/134 [00:00<?, ? examples/s]

✅ Saved chunk 18240–18374 to disk
🎉 All chunks processed and saved successfully!


In [96]:
# merge all chunks back into a single dataset
all_chunks = []
for start in range(0, total, chunk_size):
    end = min(start + chunk_size, total)
    chunk = Dataset.load_from_disk(f"{save_dir}/chunk_{start}_{end}")
    all_chunks.append(chunk)
final_dataset = concatenate_datasets(all_chunks)
# Save final merged dataset
final_dataset.save_to_disk(f"{save_dir}/final_lamini_informational_creative_dataset")
print("🎉 All chunks processed and saved successfully!")

Saving the dataset (0/1 shards):   0%|          | 0/18374 [00:00<?, ? examples/s]

🎉 All chunks processed and saved successfully!


In [99]:
# display first few rows of final dataset
final_dataset = Dataset.load_from_disk(f"{save_dir}/final_lamini_informational_creative_dataset")
final_df = final_dataset.to_pandas()
# display first 10 rows
pd.set_option('display.max_colwidth', 50)
final_df.head(64)

,chosen,rejected,input,label,factual_score,creative_score
0,To determine which aircraft is more cost-effec...,To determine which aircraft is more cost-effec...,An airline is considering adding a new route b...,factual,0.558809,0.441191
1,A bus is typically heavier than a car. This is...,A bus is generally heavier than a car. Cars co...,what is heavier a car or a bus?,factual,0.859074,0.140926
2,Predators:\n- Cheetah\n\nPrey:\n- Rabbit\n\nAl...,Cheetah - Predator\nElephant - Neither (Herbiv...,Categorize these animals as predators/prey.\nC...,factual,0.855127,0.144873
3,1. Software Developer: Software developers use...,1. Software Engineers \n2. Programmers \n3. Da...,Name 3 professions that use computers for thei...,factual,0.599251,0.400749
4,I understand that you'd like a code function t...,Here is a Python code that can perform this ta...,Find the three least common values in the give...,factual,0.685529,0.314471
...,...,...,...,...,...,...
59,Step 1: Calculate the total number of flowers ...,Step 1: Calculate the total number of flowers ...,"In a garden, there are 6 rows of flowers with ...",factual,0.857291,0.142709
60,Those are some great libraries and packages yo...,"As a responsible and helpful AI assistant, I c...",What are some JavaScript libraries or packages...,creative,0.238639,0.761361
61,Rainbow,Rainbow,"What natural phenomenon, caused by the reflect...",creative,0.402627,0.597373
62,"These are all excellent suggestions. However, ...",Thank you for sharing the details of your high...,How can I craft an attention-grabbing and dist...,creative,0.129513,0.870487


### testing bw flan base and small

In [44]:
classifier = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    device_map="auto"
)

def classify_prompt(prompt):
    # system_prompt = """Classify the following user prompt as either 'informational' or 'creative'.

    # - 'Informational' prompts seek factual, analytical, or explanatory responses — they ask for definitions, explanations, factual data, or reasoning.
    # - 'Creative' prompts request imaginative or open-ended generation — such as writing stories, lyrics, poems, roleplays, or brainstorming ideas. 
    # Prompts beginning with verbs like "write", "generate", "compose", or "create" are usually creative.

    # Examples:

    # Prompt: What is the capital of Japan?
    # Answer: informational

    # Prompt: Explain why the sky appears blue.
    # Answer: informational

    # Prompt: Generate a short story about a mythical creature.
    # Answer: creative

    # Prompt: Write a poem about time and memory.
    # Answer: creative

    # Prompt: """

    system_prompt = """Classify the following user prompt as either 'informational' or 'creative'."""

    query = f"{system_prompt}{prompt}\nAnswer:"
    output = classifier(query, max_new_tokens=5)[0]['generated_text']
    return output.strip()

Device set to use mps


In [76]:
classifier3 = pipeline(
    "text2text-generation",
    model="MBZUAI/LaMini-Flan-T5-248M",
    device_map="auto"
)

def classify_prompt3(prompt):
    # system_prompt = """Classify the following user prompt as either 'informational' or 'creative'.

    # - 'Informational' prompts seek factual, analytical, or explanatory responses — they ask for definitions, explanations, factual data, or reasoning.
    # - 'Creative' prompts request imaginative or open-ended generation — such as writing stories, lyrics, poems, roleplays, or brainstorming ideas. 
    # Prompts beginning with verbs like "write", "generate", "compose", or "create" are usually creative.

    # Examples:

    # Prompt: What is the capital of Japan?
    # Answer: informational

    # Prompt: Explain why the sky appears blue.
    # Answer: informational

    # Prompt: Generate a short story about a mythical creature.
    # Answer: creative

    # Prompt: Write a poem about time and memory.
    # Answer: creative

    # Prompt: """

    system_prompt = """Classify the following user prompt as either 'informational' or 'creative'."""

    query = f"{system_prompt}{prompt}\nAnswer:"
    output = classifier3(query, max_new_tokens=5)[0]['generated_text']
    return output.strip()

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use mps


In [52]:
classifier2 = pipeline(
    "text2text-generation",
    model="google/flan-t5-small",
    device_map="auto"
)

def classify_prompt2(prompt):
    system_prompt = """Classify the following user prompt as either 'informational' or 'creative'.

    - 'Informational' prompts seek factual, analytical, or explanatory responses — they ask for definitions, explanations, factual data, or reasoning.
    - 'Creative' prompts request imaginative or open-ended generation — such as writing stories, lyrics, poems, roleplays, or brainstorming ideas. 
    Prompts beginning with verbs like "write", "generate", "compose", or "create" are usually creative.

    Examples:

    Prompt: What is the capital of Japan?
    Answer: informational

    Prompt: Explain why the sky appears blue.
    Answer: informational

    Prompt: Generate a short story about a mythical creature.
    Answer: creative

    Prompt: Write a poem about time and memory.
    Answer: creative

    Prompt: """

    query = f"{system_prompt}{prompt}\nAnswer:"
    output = classifier2(query, max_new_tokens=5)[0]['generated_text']
    return output.strip()

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [77]:
prompt = "You need to complete the following task: Generate a few lyrics for a rap about getting up early in the morning."
# print("Using flan-t5-small:")
# print(classify_prompt2(prompt))
print("Using flan-t5-base:")
print(classify_prompt(prompt))
print("Using LaMini-Flan-T5-248M:")
print(classify_prompt3(prompt))

Using flan-t5-base:
creative
Using LaMini-Flan-T5-248M:
Creative.


In [78]:
prompt = "Generate a short story about a mythical creature."
# print("Using flan-t5-small:")
# print(classify_prompt2(prompt))
print("Using flan-t5-base:")
print(classify_prompt(prompt))
print("Using LaMini-Flan-T5-248M:")
print(classify_prompt3(prompt))

Using flan-t5-base:
creative
Using LaMini-Flan-T5-248M:
Creative.


# embeddings

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd
from tqdm.auto import tqdm

# === Config ===
MODEL = "MBZUAI/LaMini-Flan-T5-248M"
device = "cuda" if torch.cuda.is_available() else "cpu"

# === Load model & tokenizer ===
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL, torch_dtype=torch.float16 if device=="cuda" else torch.float32)
model.to(device)
model.eval()

# === Embedding function ===
@torch.inference_mode()
def get_embeddings(texts, pooling="mean", batch_size=32):
    all_embeddings = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch_texts = texts[i:i+batch_size]
        tokens = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt").to(device)
        encoder_outputs = model.get_encoder()(**tokens)
        hidden_states = encoder_outputs.last_hidden_state  # [batch, seq_len, hidden_dim]

        if pooling == "mean":
            mask = tokens.attention_mask.unsqueeze(-1)
            summed = (hidden_states * mask).sum(dim=1)
            counts = mask.sum(dim=1).clamp(min=1)
            embeddings = summed / counts
        elif pooling == "cls":
            embeddings = hidden_states[:, 0, :]
        all_embeddings.append(embeddings.cpu())
        # display one embedding set
        print(embeddings[0])
    return torch.cat(all_embeddings, dim=0)

# === Example: annotate pandas DataFrame ===
# df should have columns: "prompt", "chosen_response", "rejected_response"
# Compute embeddings for final_dataset
final_dataset = Dataset.load_from_disk(f"{save_dir}/final_lamini_informational_creative_dataset")
final_df = final_dataset.to_pandas()
# Compute embeddings
final_df["prompt_embedding"] = list(get_embeddings(final_df["input"].tolist()))
final_df["chosen_embedding"] = list(get_embeddings(final_df["chosen"].tolist()))
final_df["rejected_embedding"] = list(get_embeddings(final_df["rejected"].tolist()))

np.save("../data/dpo_informational_creative_split_lamini_version/prompt_embeddings.npy", np.stack([e.numpy() for e in final_df["prompt_embedding"]]))
np.save("../data/dpo_informational_creative_split_lamini_version/chosen_embeddings.npy", np.stack([e.numpy() for e in final_df["chosen_embedding"]]))
np.save("../data/dpo_informational_creative_split_lamini_version/rejected_embeddings.npy", np.stack([e.numpy() for e in final_df["rejected_embedding"]]))

  0%|          | 0/1 [00:00<?, ?it/s]

tensor([ 3.3375e-02,  1.7133e-02, -4.2530e-02,  7.7025e-02, -1.5892e-02,
         5.9383e-03, -2.1788e-02, -1.9217e-02, -5.5245e-02,  2.4526e-02,
        -9.3737e-03,  2.9770e-02, -1.6945e-02, -1.6878e-02, -4.9738e-02,
        -3.4885e-03,  1.4403e-02,  3.5575e-02, -6.0567e-02,  5.3105e-02,
        -3.3759e-02, -3.0285e-02, -6.0833e-02,  6.7507e-02, -1.1552e-01,
         6.4358e-03, -5.1069e-02, -1.2238e-03, -4.5296e-02, -9.7731e-02,
         2.6859e-02, -3.6568e-02, -3.9389e-02, -4.1824e-04, -2.1361e-02,
        -2.0455e-05, -2.6114e-02,  3.1479e-02, -3.1551e-02, -2.0714e-02,
        -3.4689e-02,  1.4228e-02,  2.5267e-02,  7.5687e-03, -2.4798e-02,
        -3.2461e-02,  6.1301e-02,  9.3017e-03, -2.3622e-02,  7.7027e-02,
        -1.5852e-02, -1.3035e-02,  1.7265e-02,  5.4415e-02,  1.2563e-01,
        -2.0537e-02,  6.9273e-02,  1.1727e-02, -2.4635e-02, -2.9094e-02,
        -4.5790e-02, -5.5898e-02,  9.8957e-02,  6.5286e-02,  6.1671e-02,
        -5.8087e-02,  4.5679e-02,  2.7775e-02,  7.5

  0%|          | 0/1 [00:00<?, ?it/s]

tensor([ 0.0407,  0.0488, -0.0663,  0.1052, -0.0392,  0.0223, -0.0336,  0.0194,
        -0.0438,  0.0041,  0.0136, -0.0358, -0.0806, -0.0009, -0.0632, -0.0651,
         0.0195,  0.0009,  0.0005,  0.1114, -0.0018, -0.0287, -0.0160,  0.0825,
        -0.1285,  0.0097, -0.0713, -0.0217, -0.0563, -0.0950,  0.0808, -0.0631,
         0.0253, -0.0108, -0.0132,  0.0519, -0.0041,  0.0297, -0.0218,  0.0157,
        -0.0156, -0.0212,  0.0266,  0.0269, -0.0090,  0.0257,  0.0698, -0.0474,
        -0.0425,  0.0362, -0.0301, -0.0667,  0.0626,  0.0465,  0.2007,  0.0034,
        -0.0104, -0.0366, -0.0042, -0.0176,  0.0044, -0.0953,  0.0444,  0.0354,
         0.0275,  0.0064,  0.0772, -0.0619,  0.0488, -0.0026, -0.0498,  0.0405,
        -0.1232,  0.0587, -0.1260,  0.0087, -0.0383, -0.0718,  0.0116, -0.0016,
        -0.0809,  0.0722, -0.1117, -0.0406,  0.0007, -0.0397,  0.0418, -0.0530,
        -0.0534,  0.1049, -0.0071, -0.0865,  0.0260,  0.0669,  0.0287,  0.0309,
         0.0352,  0.0276, -0.0572, -0.06

  0%|          | 0/1 [00:00<?, ?it/s]

tensor([ 0.0407,  0.0488, -0.0663,  0.1052, -0.0392,  0.0223, -0.0336,  0.0194,
        -0.0438,  0.0041,  0.0136, -0.0358, -0.0806, -0.0009, -0.0632, -0.0651,
         0.0195,  0.0009,  0.0005,  0.1114, -0.0018, -0.0287, -0.0160,  0.0825,
        -0.1285,  0.0097, -0.0713, -0.0217, -0.0563, -0.0950,  0.0808, -0.0631,
         0.0253, -0.0108, -0.0132,  0.0519, -0.0041,  0.0297, -0.0218,  0.0157,
        -0.0156, -0.0212,  0.0266,  0.0269, -0.0090,  0.0257,  0.0698, -0.0474,
        -0.0425,  0.0362, -0.0301, -0.0667,  0.0626,  0.0465,  0.2007,  0.0034,
        -0.0104, -0.0366, -0.0042, -0.0176,  0.0044, -0.0953,  0.0444,  0.0354,
         0.0275,  0.0064,  0.0772, -0.0619,  0.0488, -0.0026, -0.0498,  0.0405,
        -0.1232,  0.0587, -0.1260,  0.0087, -0.0383, -0.0718,  0.0116, -0.0016,
        -0.0809,  0.0722, -0.1117, -0.0406,  0.0007, -0.0397,  0.0418, -0.0530,
        -0.0534,  0.1049, -0.0071, -0.0865,  0.0260,  0.0669,  0.0287,  0.0309,
         0.0352,  0.0276, -0.0572, -0.06

## Composition - 

- truthfulQA - ~900 factual truth testing questions (with correct and incorrect)
- flan_v2_niv2 - ~15k prompts with correct answer
- evol_instruct - good mix, but no correct/incorrect. 
- sharegpt - ~20k real prompts and responses. no correct/incorrect.
- ultrachat - variety of qs. no correct/incorrect. 